# IMPORTING PACKAGES

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.0 MB/s eta 0:00:00


In [4]:
#Importing the required packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import manifold, metrics
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier, ElasticNet
from matplotlib.pyplot import plot, show, savefig, xlim, figure, ylim, legend, boxplot, setp, axes
from sklearn.model_selection import RepeatedKFold, cross_val_score, train_test_split, KFold
from numpy import mean, std
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from math import *
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import optuna
from sklearn.svm import SVR

# PREPROCESSING

In [5]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
	iris.data,
	iris.target,
	test_size=0.2,
	random_state=42
)

In [6]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

def objective(trial):
    # Define hyperparameters
    c= trial.suggest_float("C", 1e-10, 1e10, log=True)
    kernel = trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"])

    # Create and train the model
    svm = SVC(C=c, kernel=kernel, random_state=42)
    svm.fit(X_train, y_train)

    # Evaluate the model
    y_pred = svm.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

In [7]:
import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=2)

[I 2024-08-29 07:15:03,120] A new study created in memory with name: no-name-86d1fb34-dd12-413a-83d3-2e2d100cfbf9
[I 2024-08-29 07:15:03,154] Trial 0 finished with value: 0.3 and parameters: {'C': 2.4405085163610645e-10, 'kernel': 'poly'}. Best is trial 0 with value: 0.3.
[I 2024-08-29 07:15:03,191] Trial 1 finished with value: 0.3 and parameters: {'C': 0.0006353774530989517, 'kernel': 'linear'}. Best is trial 0 with value: 0.3.


In [8]:
print("Best trial:", study.best_trial.number)
print("Best accuracy:", study.best_trial.value)
print("Best hyperparameters:", study.best_params)

Best trial: 0
Best accuracy: 0.3
Best hyperparameters: {'C': 2.4405085163610645e-10, 'kernel': 'poly'}


In [9]:
# Using a Pruner to End Experiments Early
from sklearn.model_selection import cross_val_score
from optuna.pruners import SuccessiveHalvingPruner

def objective(trial):
    c = trial.suggest_float("C", 1e-10, 1e10, log=True)
    kernel = trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"])

    svm = SVC(C=c, kernel=kernel, random_state=42)

    for step in range(5):
        score = cross_val_score(svm, X_train, y_train, cv=5, scoring='accuracy').mean()
        trial.report(score, step)

        # Check for pruning
        if trial.should_prune():
            raise optuna.TrialPruned()

    return score

pruner = SuccessiveHalvingPruner(min_resource=1, reduction_factor=2, min_early_stopping_rate=0)
study = optuna.create_study(pruner=pruner, direction='maximize')
study.optimize(objective, n_trials=20)

[I 2024-08-29 07:15:05,337] A new study created in memory with name: no-name-0b6e86c6-c63d-4c10-874e-c1b508d8d0b3
[I 2024-08-29 07:15:05,608] Trial 0 finished with value: 0.9416666666666667 and parameters: {'C': 1283.0316887815272, 'kernel': 'rbf'}. Best is trial 0 with value: 0.9416666666666667.
[I 2024-08-29 07:15:05,737] Trial 1 pruned. 
[I 2024-08-29 07:15:10,189] Trial 2 finished with value: 0.9416666666666668 and parameters: {'C': 176743.72326213014, 'kernel': 'poly'}. Best is trial 2 with value: 0.9416666666666668.
[I 2024-08-29 07:15:10,372] Trial 3 finished with value: 0.95 and parameters: {'C': 741.9444915393445, 'kernel': 'linear'}. Best is trial 3 with value: 0.95.
[I 2024-08-29 07:15:10,529] Trial 4 pruned. 
[I 2024-08-29 07:15:10,673] Trial 5 pruned. 
[I 2024-08-29 07:15:13,135] Trial 6 finished with value: 0.9416666666666668 and parameters: {'C': 27746.54199122427, 'kernel': 'poly'}. Best is trial 3 with value: 0.95.
[I 2024-08-29 07:15:13,205] Trial 7 pruned. 
[I 2024-0

In [10]:
def print_trial_info(study, trial):
    print(f"Trial {trial.number}: Value = {trial.value}")

In [11]:
# Passing Our Callback Into the Study
def objective(trial):
    c = trial.suggest_float("C", 1e-10, 1e10, log=True)
    kernel = trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"])

    svm = SVC(C=c, kernel=kernel, random_state=42)
    svm.fit(X_train, y_train)

    y_pred = svm.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20, callbacks=[print_trial_info])

[I 2024-08-29 07:15:26,141] A new study created in memory with name: no-name-cba69e95-e07c-4723-b5be-3a2b3c6fe51a
[I 2024-08-29 07:15:26,159] Trial 0 finished with value: 0.9666666666666667 and parameters: {'C': 7.306502835118254, 'kernel': 'linear'}. Best is trial 0 with value: 0.9666666666666667.
[I 2024-08-29 07:15:26,179] Trial 1 finished with value: 0.03333333333333333 and parameters: {'C': 2565.4750860140693, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.9666666666666667.


Trial 0: Value = 0.9666666666666667
Trial 1: Value = 0.03333333333333333


[I 2024-08-29 07:15:27,120] Trial 2 finished with value: 0.9666666666666667 and parameters: {'C': 9846817912.676752, 'kernel': 'poly'}. Best is trial 0 with value: 0.9666666666666667.


Trial 2: Value = 0.9666666666666667


[I 2024-08-29 07:15:27,470] Trial 3 finished with value: 1.0 and parameters: {'C': 520070848.2219913, 'kernel': 'linear'}. Best is trial 3 with value: 1.0.
[I 2024-08-29 07:15:27,490] Trial 4 finished with value: 1.0 and parameters: {'C': 6391.16086358864, 'kernel': 'rbf'}. Best is trial 3 with value: 1.0.
[I 2024-08-29 07:15:27,505] Trial 5 finished with value: 0.03333333333333333 and parameters: {'C': 126.48991673685185, 'kernel': 'sigmoid'}. Best is trial 3 with value: 1.0.
[I 2024-08-29 07:15:27,526] Trial 6 finished with value: 0.03333333333333333 and parameters: {'C': 440911.1699320913, 'kernel': 'sigmoid'}. Best is trial 3 with value: 1.0.
[I 2024-08-29 07:15:27,545] Trial 7 finished with value: 0.9333333333333333 and parameters: {'C': 375200.19727813726, 'kernel': 'rbf'}. Best is trial 3 with value: 1.0.
[I 2024-08-29 07:15:27,560] Trial 8 finished with value: 0.3 and parameters: {'C': 0.510000114871184, 'kernel': 'sigmoid'}. Best is trial 3 with value: 1.0.


...

Trial 9: Value = 1.0
Trial 10: Value = 0.3
Trial 11: Value = 0.3
Trial 12: Value = 0.9333333333333333
Trial 13: Value = 0.3
Trial 14: Value = 1.0
Trial 15: Value = 0.3
Trial 16: Value = 0.9333333333333333


[I 2024-08-29 07:15:28,675] Trial 17 finished with value: 1.0 and parameters: {'C': 4072664782.9694495, 'kernel': 'linear'}. Best is trial 3 with value: 1.0.


Trial 17: Value = 1.0


[I 2024-08-29 07:15:29,715] Trial 18 finished with value: 0.9666666666666667 and parameters: {'C': 177224.70866817242, 'kernel': 'poly'}. Best is trial 3 with value: 1.0.
[I 2024-08-29 07:15:29,758] Trial 19 finished with value: 0.6666666666666666 and parameters: {'C': 0.04732706841543017, 'kernel': 'rbf'}. Best is trial 3 with value: 1.0.


Trial 18: Value = 0.9666666666666667
Trial 19: Value = 0.6666666666666666


In [12]:
import optuna

storage_url = "sqlite:///example.db"
study_name = "svm_optimization"

study = optuna.create_study(storage=storage_url, study_name=study_name, direction='maximize', load_if_exists=True)
study.optimize(objective, n_trials=20)

[I 2024-08-29 07:15:32,732] A new study created in RDB with name: svm_optimization
[I 2024-08-29 07:15:33,156] Trial 0 finished with value: 0.3 and parameters: {'C': 1.5194600396276257, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.3.
[I 2024-08-29 07:15:33,453] Trial 1 finished with value: 0.3 and parameters: {'C': 0.00030809059200042715, 'kernel': 'linear'}. Best is trial 0 with value: 0.3.
[I 2024-08-29 07:15:33,693] Trial 2 finished with value: 0.9333333333333333 and parameters: {'C': 69416.23700194083, 'kernel': 'rbf'}. Best is trial 2 with value: 0.9333333333333333.
[I 2024-08-29 07:15:33,888] Trial 3 finished with value: 0.2 and parameters: {'C': 2.7901235079920457, 'kernel': 'sigmoid'}. Best is trial 2 with value: 0.9333333333333333.
[I 2024-08-29 07:15:34,106] Trial 4 finished with value: 0.3 and parameters: {'C': 1.3439468332331257e-10, 'kernel': 'linear'}. Best is trial 2 with value: 0.9333333333333333.
[I 2024-08-29 07:15:34,385] Trial 5 finished with value: 1.0 and p

In [13]:
# Resuming an Existing Study
study_resume = optuna.create_study(storage=storage_url, study_name=study_name, direction='maximize', load_if_exists=True)
study_resume.optimize(objective, n_trials=20)

[I 2024-08-29 07:15:39,476] Using an existing study with name 'svm_optimization' instead of creating a new one.
[I 2024-08-29 07:15:39,608] Trial 20 finished with value: 1.0 and parameters: {'C': 34.12180337483064, 'kernel': 'linear'}. Best is trial 5 with value: 1.0.
[I 2024-08-29 07:15:39,712] Trial 21 finished with value: 1.0 and parameters: {'C': 49188.990515318, 'kernel': 'linear'}. Best is trial 5 with value: 1.0.
[I 2024-08-29 07:15:40,002] Trial 22 finished with value: 1.0 and parameters: {'C': 3999381.310473465, 'kernel': 'linear'}. Best is trial 5 with value: 1.0.
[I 2024-08-29 07:15:40,105] Trial 23 finished with value: 1.0 and parameters: {'C': 1641.0368134209662, 'kernel': 'linear'}. Best is trial 5 with value: 1.0.
[I 2024-08-29 07:15:40,205] Trial 24 finished with value: 1.0 and parameters: {'C': 0.0832152957710041, 'kernel': 'linear'}. Best is trial 5 with value: 1.0.
[I 2024-08-29 07:15:40,308] Trial 25 finished with value: 0.9333333333333333 and parameters: {'C': 1065

In [14]:
saved_study = optuna.load_study(study_name=study_name, storage=storage_url)
print("Best trial:", saved_study.best_trial.number)
print("Best value:", saved_study.best_trial.value)
print("Best hyperparameters:", saved_study.best_params)

Best trial: 5
Best value: 1.0
Best hyperparameters: {'C': 519.5445430973323, 'kernel': 'linear'}


In [15]:
# Plotting Optimization History
import optuna.visualization as vis

optimization_history_plot = vis.plot_optimization_history(study)
optimization_history_plot.show()

In [16]:
# Plotting Parameter Importance
param_importance_plot = vis.plot_param_importances(study)
param_importance_plot.show()

In [17]:
# Feature Selection Using Optuna
import optuna
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Load the dataset and split it into training and testing sets
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

# Define the objective function
def objective(trial):
    k = trial.suggest_int("k", 1, X_train.shape[1])
    selected_features = SelectKBest(f_classif, k=k).fit_transform(X_train, y_train)

    n_estimators = trial.suggest_int("n_estimators", 10, 100)
    max_depth = trial.suggest_int("max_depth", 2, 10, log=True)

    classifier = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    classifier.fit(selected_features, y_train)

    selected_features_test = SelectKBest(f_classif, k=k).fit_transform(X_test, y_test)
    y_pred = classifier.predict(selected_features_test)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=2)

[I 2024-08-29 07:16:24,084] A new study created in memory with name: no-name-4ee49bf2-8a17-4589-ae78-5ef6107aeaf4
[I 2024-08-29 07:16:24,280] Trial 0 finished with value: 0.3333333333333333 and parameters: {'k': 1, 'n_estimators': 79, 'max_depth': 3}. Best is trial 0 with value: 0.3333333333333333.
[I 2024-08-29 07:16:24,326] Trial 1 finished with value: 0.3333333333333333 and parameters: {'k': 1, 'n_estimators': 15, 'max_depth': 2}. Best is trial 0 with value: 0.3333333333333333.
